# FDMBuilder - The Extras

If you've arrived here, hopefully you're already comfortable with the basics of the `FDMBuilder` API and have played aroudnd with the `FDMTable` and `FDMDataset` classes a bit. If not, take a look at `fdm_builder_basics_tutorial` and then come back here when you're ready.

This tutorial will cover the extra methods and helpers that can be found in the FDMBuilder library. Hopefully, they can help you speed up your FDM building workflow even more. We'll start by taking a look at each of the tools/methods/helpers first, followed by a (hopefully) motivating example that will demonstrate how these tools/methods/helpers can be used "in the wild".

As with the basics tutorial, we'll start by loading the `FDMBuilder` libraries and create a `DATASET_ID` variable for your test dataset, to make the example code below a little easier to run. Same as before, don't specify a dataset with any tables in it that you'll miss, because we're about to delete them! Replace the `YOUR DATASET HERE` text with the id of your test dataset, and then run the below cell:

In [ ]:
from FDMBuilder.FDMTable import *
from FDMBuilder.FDMDataset import *
from FDMBuilder.testing_helpers import *

### !!REPLACE THIS TEXT!! ###

DATASET_ID = "CY_SAM_TEST"

###

# Leave this bit alone!
if check_dataset_exists(DATASET_ID):
    clear_dataset(DATASET_ID)
    print("Good to go!")
else:
    print("#" * 33 + " PROBLEM!! " + 33 * "#" + "\n")
    print("Something doesn't look right. Check you spelled everything correctly,\n" 
          "your dataset has been created in GCP, and you have the right permisssions\n")
    print("#" * 80)

## BigQuery Cell Magics

The first tool isn't anything the FDM pipeline can take credit for. Packaged in the python bigquery library is a "cell magic", that allows you to run pure SQL queries directly from a Jupyter notebook cell. A cell magic is a little bit of syntax that changes or adds extra functionality to a notebook cell - the general syntax of a cell magic is `%%magic-name`, so the bigquery cell magic is `%%bigquery`. Add that to the top of a cell, write your SQL below, and juypter/python will do the rest. Give the below a try:

In [ ]:
%%bigquery
SELECT *
FROM `CY_FDM_MASTER.person`
LIMIT 10

Easy. 

For those familiar with the pandas library, you can store the results of your query as a `pandas.DataFrame` by naming it immediately after the `%%bigquery` magic. So the following cell runs the same query as above, and stores the result in `eg_df`:

In [ ]:
%%bigquery eg_df
SELECT *
FROM `CY_FDM_MASTER.person`
LIMIT 10

You can then run the following to take a look at the contents of `eg_df`:

In [ ]:
eg_df

If so inclined, you can run and document your SQL pipelines in a notebook by using the above cell magics, and then documenting your work in markdown text cells (like this). Be sure to only stick SQL cells marked with the `%%bigquery` magic - everything inside these cells is interpreted as SQL, so you'll get some pretty colourful errors if you try sticking python in there too.

Now on to the extra bits of the FDM pipeline. 

## FDMTable Helpers

The `FDMTable` comes with a bunch of extra "methods" that quicken up some of the more "fiddly" bits of the BigQuery SQL environment. We'll be using the last of the test tables `test_table_3` to try out these new helpers. By now, it hopefully won't be a shock to see us begin by initialising an `FDMTable` with the location of `test_table_3` and our test dataset:

In [ ]:
test_table_3 = FDMTable(
    source_table_id = "CY_FDM_BUILDER_TESTS.test_table_3",
    dataset_id=DATASET_ID
)

### copy_table_to_dataset

In your adventures with the `.build()` process, you may have noticed that the first stage is copying the source table into the FDM dataset. This is a requirement before you start manipulating a table - you don't want to be messing about with the original copy of the data in GCP. Oh no. We need a pristine copy of the original source data, in case anything goes wrong.

Based on this, none of the methods below will work if you don't first make a copy of your source table in your FDM dataset - otherwise there's no table in GCP for your `FDMTable` to manipulate. You can quickly create a copy of the source table using the `copy_table_to_dataset` method like so:

In [ ]:
test_table_3.copy_table_to_dataset()

As you'd expect, if you pop over to your GCP SQL workspace, you'll find a fresh copy of `test_table_3` in your test dataset. Pretty simple. Now you can begin playing around with some of the helper functions.

### head

If you want a quick reminder of the contents of your `FDMTable` you can call the `head` function - by default it will return a dataframe with the first 10 rows of your table, like so:

In [ ]:
test_table_3.head()

If, for some reason, you'd like to see a different number of rows, you can use the `n` argument like so:

In [ ]:
test_table_3.head(n=5)

simple.

### rename_columns

Next is `rename_columns` - you'll be shocked to hear this renames columns in your table. This is surprisingly awkward to do in BigQuery SQL syntax. Hopefully you'll find this a little easier. The logic is as follows:

`rename_columns` takes one argument - a python "dictionary". Dictionaries are a data type in python that comine a set of "keys" with a set of "values". They look like this:

```
    example_dict = {
        "key_1": "value_1",
        "key_2": "value_2",
        "key_3": "value_3",
        ...
    }
```

Dictionaries are defined inside curly braces - `{}` - inside which are "keys" and "values" separated by a colon - `:` - and each key-value pair is separated by a comma - `,`. The input to `rename_columns` is a dictionary where each key is an existing column name that you want to change, and each value is the new name, like so:

```
    example_rename_columns_input = {
        "old_name_1": "new_name_1",
        "old_name_2": "new_name_2",
        "old_name_3": "new_name_3",
        ...
    }
```

So we can rename the `some_data` column in our `test_table_3` to `some_new_data` by running the following code cell:

In [ ]:
test_table_3.rename_columns({"some_data": "some_new_data"})

and we can check that worked using our new found `head` function:

In [ ]:
test_table_3.head()

### add_column

Next on the list is `add_column`. Shockingly, this addes a new column to our Table. It takes one argument, a string that should look like one column of your standard `SELECT` statement. So, for example:

    "some_new_data * 100 AS some_new_data_x_100"
    "LENGTH(education_reference) AS ed_ref_length"
    "LOWER(educatoin_reference) AS lower_case_ed_ref"

If you could stick it at the start of a select statement, it'll work in `add_column`. If you're wondering what any of the above will do, given them a try in the next cell:

In [ ]:
test_table_3.add_column("some_new_data * 100 AS new_data_x_100")

test_table_3.head()

### drop_column

This one couldn't be easier - `drop_column` takes one argument, a column name, and drops/deletes the named column. Give it a try:

In [ ]:
test_table_3.drop_column("some_new_data")

test_table_3.head()

### quick_build

When you were running through the basics of the FDM API, you may have thought the `build` process was a bit lengthy - what with all the long-winded explanations and requests for input. That's where `quick_build` comes in. It's basically a more "programatic" way of completing the build process. It takes the training wheels off, so to speak, and in doing so makes the process a lot "snappier".

`quick_build` takes up to five arguments:

* `fdm_start_date_cols`: Either a string or a list, detailing the columns that contain the start date information. If the start date is found in a single column with a datetime or a string that can easily be parsed, then the input would be a string with the column name. If the start date is in multiple columns with individual year, month and day, the input would be a list with the column names - or, a static value for one or more of the year/month/day. If we think back to the test examples, for `test_table_1` we would use `"start_date"` and for `test_table_2` we would input `["start_year", "start_month", "15"]`
* `fdm_start_date_format`: This is one of `"YMD"`/`"YDM"`/`"DMY"`/`"MDY"`. Hopefully fairly self explanatory. Simply the date format of the start date data. This is required both if you input a single column or multiple columns - so it doesn't matter what order you input your `fdm_start_date_cols` provided you correctly specify the date format.
* `fdm_end_date_cols`: This is an optional argument, depending on the need for an end date in the source data you're FDMing. It takes exactly the same input format as the `fdm_start_date_cols` argument, so no need to go over that again.
* `fdm_end_date_format`: Again, an optional argument depending on the presence of an end date, with the same input specification as the `fdm_start_date_format`
* `verbose`: By default this is set to `True`, and controls the console output while the `quick_build` process is running. When set to `True`, the console will output text telling you what stage the script has reached. If set to `False`, the console output is surpressed.

That may seem a little daunting to take in all at once, but the process is really pretty simple once you get started. We'll save working examples for the moment, as we'll see plenty when we take a look at an example workflow below.

### recombine

You'll hopefully recall that, as part of the `FDMDataset`'s `build` process, the tool splits any entries that are found to have "problems" into separate tables. This is an important part of completing an FDM dataset, but presents an issue if you then want to start manipulating the source data after the build - you have two separate tables that contain the source data! This could quickly lead to errors, particularly if you want to correct any of the problems identified during the FDM build.

`recombine` is a method designed to resolve this issue. If you want to start manipulating a table that has been split from it's problematic entries, you must first `recombine` it - stich the two tables back together. The method itself couldn't be easier to use - simply call `your_table.recombine()` on a table that has an associated "fdm_problems" table, and the script does the rest. 

You'll find that if you try and use any of the above helpers on a table that has a separate "fdm_problems" table or you try to build an FDM from a dataset containing "fmd_problems" tables, the method will return an error and ask you to first recombine the problem entries before continuning.  To help in any efforts to correct problem entries, the "problem" column from the associated "fdm_problems" table is kept after using `recombine`, and is `NULL` for any entries that don't have an associated problem. 

It would be a bit of a faff setting up an example here, but there will be examples of `recombine` in the workflow below.

## Other Helpers

Before we take a look at a more involved example workflow, a quick mention of a couple of functions we've already seen but have glossed over until now. These aren't methods attached to either the `FDMTable` or `FDMDataset` classes, but stand-alone functions in their own right. They're all very simple to use:

### check_dataset_exists / check_table_exists

Both do what they say on the tin - either checking a table or a dataset exists! Simply stick the id of a table or dataset in the function, and it will return either `True` or `False` depending on the existence of the named table/dataset:


In [ ]:
check_dataset_exists("CY_FDM_MASTER")

In [ ]:
check_table_exists("this_table.doesn't.exist")

### clear_dataset

Again, does pretty much what it says on the tin. `clear_dataset` will remove every table from the dataset you point it at. Obviously, to be used with caution! We'll need to clear out our test dataset before we can start with the example workflow below, so let's do just that:

In [ ]:
clear_dataset(DATASET_ID)

## Example FDM Building Workflow

Righty, we've reached the now much talked about example workflow. In this example, we'll assume we're building an FDM from scratch, using the tables in `CY_FDM_BUILDER_TESTS`. The process will be similar to that of the basics tutorial, but a little more "programatic" and with less navigating backwards and forwards between jupyter and GCP.

A quick note - you should expect to see some errors when going through this workflow. Indeed errors are a normal part of programming in any language! The FDM tools have been designed to be (relatively) robust to various missteps, so you should see an informative error message if you something one of the tools doesn't like. At any rate, if you see an error here, don't panic - it's supposed to be there. Just give the error message a quick read (the main message will be at the bottom of the error readout) and continue with the tutorial - it will (hopefully) keep you abreast of what's happening. 

We'll start, as we did before, by building each of the individual tables in our test datset. Let's start with `test_table_1`: 

In [ ]:
test_table_1 = FDMTable(
    source_table_id = "CY_FDM_BUILDER_TESTS.test_table_1",
    dataset_id = DATASET_ID
)

let's take a quick look at the data in `test_table_1`

In [ ]:
test_table_1.head()

and there's our first error! You'll remember that we need a copy of our source dataset in our working datset, otherwise we cann't do anything with it. If you don't, reading the error message above should hopefully serve as a reminder. Let's make a copy and try that again:

In [ ]:
test_table_1.copy_table_to_dataset()
test_table_1.head()

Great. `test_table_1` looks like it's already in a perfect state to complete the FDM build process. We'll have our first go with the `quick_build` method. Remember, we just need to point the method to the column(s) that contain the start dates and, if necessary, the end dates. In this case, there's just the one column that contains start dates:

In [ ]:
test_table_1.quick_build(
    fdm_start_date_cols="start_date",
    fdm_start_date_format="DMY"
)

and that's the first table built. You can see, the quick build method still gives you some guidance as to what's happening - it's just a lot less verbose and fiddly than the basic `build` method.

Now we'll move on to `test_table_2`. As before, we'll initialise an `FDMTable` for `test_table_2`, copy the source data across to our dataset, and take a look at what it contains:

In [ ]:
test_table_2 = FDMTable(
    source_table_id = "CY_FDM_BUILDER_TESTS.test_table_2",
    dataset_id = DATASET_ID
)
test_table_2.copy_table_to_dataset()
test_table_2.head()

You might already notice an issue that might make the build process difficult. If not, no worries - the `quick_build` method is pretty good at pointing them out to you. 

Let's have a go at building `test_table_2`. This time we have a start and end date, with years and months in separate columns, and no day information. You'll remember we tell the `FDMBuilder` how to find these dates by putting them in a list, and we can use static values for any missing years/months/days:

In [ ]:
test_table_2.quick_build(
    fdm_start_date_cols = ["start_year", "start_month", "15"],
    fdm_start_date_format = "YMD",
    fdm_end_date_cols = ["end_year", "end_month", "15"],
    fdm_end_date_format = "YMD"
)

As suspected, the `quick_build` process can't find an identifier column. There are digests to be found in `test_table_2`, they're just under a different name `wrong_digest`. We'll rename that column:

In [ ]:
test_table_2.rename_columns({"wrong_digest":"digest"})

... and then have another go:

In [ ]:
test_table_2.quick_build(
    fdm_start_date_cols = ["start_year", "start_month", "15"],
    fdm_start_date_format = "YMD",
    fdm_end_date_cols = ["end_year", "end_month", "15"],
    fdm_end_date_format = "YMD"
)

fab.

Lastly we have `test_table_3`:

In [ ]:
test_table_3 = FDMTable(
    source_table_id = "CY_FDM_BUILDER_TESTS.test_table_3",
    dataset_id = DATASET_ID
)
test_table_3.copy_table_to_dataset()
test_table_3.head()

this one looks a little more tricky! You've probably noticed that:

* there isn't a recognised identifier in this table
* the start and end dates seem to be stored in one column in quite an unhelpful format

The lack of id column should hopefully now be an easy one to fix - the "education_references" are actually EDRNs, so we just need to rename that column: 

In [ ]:
test_table_3.rename_columns({"education_reference":"EDRN"})

The dates are a little more tricky. We could really do to split up all the information in that one column into separate columns. The `add_column` helper is our friend here, along with the string functions in the BigQuery SQL documentaion:

https://cloud.google.com/bigquery/docs/reference/standard-sql/string_functions

We can use the `SPLIT` SQL function to split a string into a list, splitting at a specified character. Let's use the `add_column` method to try splitting the `examination_period` at the "-" character and see what happens:

In [ ]:
test_table_3.add_column("SPLIT(examination_period, '-') AS split_exam_period")
test_table_3.head()

That seems to be a move in the right direction, but we could really use the start and end date information in separate columns - we can use the `OFFSET` SQL function to select a specific item in a list like so:

In [ ]:
test_table_3.add_column("split_exam_period[OFFSET(0)] AS start_date")
test_table_3.add_column("split_exam_period[OFFSET(1)] AS end_date")
test_table_3.head()

That's more like it. We might now decide to get rid of our `split_exam_period` column to make things a little cleaner:

In [ ]:
test_table_3.drop_column("split_exam_period")
test_table_3.head()

We still have an issue - the start and end dates aren't full dates in their own right. We either need to add a day to each of the dates, or further divide the start and end into year and month columns. The former option is a bit fiddly, so lets split the `start_date` and `end_date` columns into `start_year`/`start_month`/`end_year`/`end_month` columns:

In [ ]:
test_table_3.add_column("SPLIT(start_date, '/')[OFFSET(0)] AS start_month")
test_table_3.add_column("SPLIT(start_date, '/')[OFFSET(1)] AS start_year")
test_table_3.add_column("SPLIT(end_date, '/')[OFFSET(0)] AS end_month")
test_table_3.add_column("SPLIT(end_date, '/')[OFFSET(1)] AS end_year")
test_table_3.head()

Great! That's everything we need to build our FDMTable - so let's do just that:

In [ ]:
test_table_3.quick_build(
    fdm_start_date_cols = ["start_year", "start_month", "15"],
    fdm_start_date_format = "YMD"
)

You might have noticed we left out the end date columns there. That was deliberate, to demonstrate that the `quick_build` process can be run iteratively. If you forget something, just re-run `quick_build` with the required ammendments and it will pick up where it left off:

In [ ]:
test_table_3.quick_build(
    fdm_start_date_cols = ["start_year", "start_month", "15"],
    fdm_start_date_format = "YMD",
    fdm_end_date_cols = ["end_year", "end_month", "15"],
    fdm_end_date_format="YMD"
)

You'll note the `quick_build` script keeps any existing work it's already done - so the `person_id` and `fdm_start_date` columns were kept as they are. If you wanted the script to re-build either of these columns, just drop them and then re-start the process again:

In [ ]:
test_table_3.drop_column("person_id")
test_table_3.quick_build(
    fdm_start_date_cols = ["start_year", "start_month", "15"],
    fdm_start_date_format = "YMD",
    fdm_end_date_cols = ["end_year", "end_month", "15"],
    fdm_end_date_format="YMD"
)

And with that, all of the tables have been built, and we're ready to build the FDMDataset. This bit is exactly the same as it was when we covered it in the basics:

In [ ]:
dataset = FDMDataset(
    dataset_id = DATASET_ID
)
dataset.build()

It seems like there were a good few issues with each of the tables - let's take a look some of the problems from `test_table_1`. The `FDMTable` object for `test_table_1` will now only point to the non-problem entries, so we'll have to use plain old SQL to look at the problem table:

(you'll need to replace YOUR DATASET HERE with your dataset_id - unfortunately python can't be used in `%%bigquery` cell magics)

In [ ]:
%%bigquery
SELECT * FROM `CY_SAM_TEST.test_table_1_fdm_problems`

Let's say, for example, that we know the missing fdm_start_dates were caused by a problem on 15th Jaunary 2020. We could correct that and recover the entries that were removed. 

To do that, we first have to `recombine` `test_table_1`, that's easily done as follows:

In [ ]:
test_table_1.recombine()
test_table_1.head()

The entries in `test_table_1_fdm_problems` have now been added back into `test_table_1`, but the problems column has been retained to make any corrections easier to carry out. We can then correct our null entries. Unfortunatley the FDM tools aren't sophisticated enough to carry out this sort of operation, but we can use SQL again:

In [ ]:
%%bigquery
UPDATE `yhcr-prd-phm-bia-core.CY_SAM_TEST.test_table_1`
SET start_date = "15-January-2020"
WHERE fdm_problem = "Entry has no fdm_start_date"

Now, we'll need to re-build the `fdm_start_date` column so it contains the new dates we've added. To do that, we drop the existing `fdm_start_date` column and then re-run `build` like so:

In [ ]:
test_table_1.drop_column("fdm_start_date")
test_table_1.quick_build(
    fdm_start_date_cols="start_date",
    fdm_start_date_format="DMY"
)

and then we can re-build the FDM dataset:

In [ ]:
dataset.build()

That about does it for this tutorial. Hopefully you can see how the various tools and helpers we've introduced here can be strung together to make a pretty clean workflow, that'll speed up developing an FDM even further.